Here deterministic strategies: LO, LI, and RI.
For ω_n (I ... I)_m define n == m for, where n is [2, 30]

# Import the dependencies

In [1]:
import sys
from tqdm import tqdm

sys.path.append("../")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *

Lambda = Abstraction
App = Application

# Defining test term:
    ω_n (I ... I)_m

In [2]:
def i_term():
    x = Var()
    x_ = Atom(x)
    return Lambda(x, x_)


def omega_term(n: int):
    x = Var()
    x_ = Atom(x)
    body_app_term = x_
    for _ in range(n - 1):
        body_app_term = App(body_app_term, x_)
    return Lambda(x, body_app_term)


def multi_i_term(m: int):
    app_term = i_term()
    for _ in range(m - 1):
        app_term = App(app_term, i_term())
    return app_term


def omega_multi_i_term(n: int, m: int):
    return App(omega_term(n), multi_i_term(m))

In [3]:
def run_experiment(strategy=RandomStrategy(), count_experiments=20, nm_pairs=None):
    steps_by_strategy = []

    for nm_pairs_list in tqdm(nm_pairs):
        steps_list = []
        for n, m in nm_pairs_list:
            norm_steps = 0
            for _ in range(count_experiments):
                term = omega_multi_i_term(n, m)
                _, norm_steps_ = term.normalize(strategy, is_limited=False)
                norm_steps += norm_steps_
            steps_list.append(norm_steps / count_experiments)
        steps_by_strategy.append(steps_list)

    return steps_by_strategy

# Run through strategies
## RandomInner

In [4]:
nm_pairs_dt = []
for n in range(2, 30 + 2):
    nm_pairs_dt.append([(n, m) for m in range(2, 30 + 2)])

In [5]:
random_steps = run_experiment(strategy=RandomInnerStrategy(), count_experiments=10 , nm_pairs=nm_pairs_dt)

100%|██████████| 30/30 [05:02<00:00, 10.09s/it]


In [8]:
import plotly.graph_objects as go
import numpy as np

N = 50
fig = go.Figure(data=[go.Mesh3d(x=np.array([n for n in range(2, 30 + 2) for m in range(2, 30 + 2)], dtype="float32"),
                                y=np.array([m for n in range(2, 30 + 2) for m in range(2, 30 + 2)], dtype="float32"),
                                z=np.array(random_steps, dtype="float32").reshape((900,)),
                                opacity=0.75,
                                color='rgba(244,22,100,0.6)',
                                hovertemplate=f"n=%{{x:.2f}}<br>m=%{{y:.2f}}<br>steps=%{{z:.2f}}",
                                )])
# fig.update_layout(scene=dict(xaxis_showspikes=False,
#                              yaxis_showspikes=False))

fig.update_layout(
    scene=dict(
        xaxis=dict(
            backgroundcolor="rgb(200, 200, 230)",
            gridcolor="white",
            showbackground=True,
            zerolinecolor="white", ),
        yaxis=dict(
            backgroundcolor="rgb(230, 200,230)",
            gridcolor="white",
            showbackground=True,
            zerolinecolor="white"),
        zaxis=dict(
            backgroundcolor="rgb(230, 230,200)",
            gridcolor="white",
            showbackground=True,
            zerolinecolor="white", ),
        xaxis_title='n',
        yaxis_title='m',
        zaxis_title='steps',
    ),
    width=1000,
    height=800,
    margin=dict(r=20, l=10, b=10, t=10))

fig.show()